In [177]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
import numpy as np
import pandas as pd


In [179]:
df = pd.read_csv("../../data/AmesHousing.csv")


In [180]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()


In [181]:
k = 10000

In [182]:
raw_y = df["SalePrice"].copy()
X = df.drop("SalePrice", axis=1).copy()

In [183]:
X = X.head(k)
raw_y = raw_y.loc[X.index].copy()

In [184]:
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

In [185]:
y = (raw_y >= raw_y.quantile(0.85)).astype(int).copy()

In [186]:
X[cat_cols] = X[cat_cols].astype("category")

In [187]:
from prism import *

In [191]:
%autoreload 2
sorter = CategoricalSorter(scoring_function=woe_scoring_func)
encoder = SortedOrdinalEncoder(sorter)
bin_calculator = NBinsCalculator(strategy=FreedmanDiaconisStrategy())
discretizer = PandasQCutDiscretizer(bin_calculator=bin_calculator)
evaluator = BinaryRuleEvaluator(score_strategies=[WoEScore(), RecallScore(), PrecisionScore()])
rule_filter = BinaryRuleFilter(min_recall=0.1)
prism = PrismRules(encoder, discretizer, evaluator)
rules = prism.find_rules(X, y, rule_filter, max_depth=3)

In [192]:
rules

[(`Overall Qual` > 8.0),
 (`Overall Qual` > 8.0) and (`Lot Frontage` <= 107.0),
 (`Overall Qual` <= 8.0) and (`Gr Liv Area` > 2463.0999999999995),
 (`Overall Qual` <= 8.0) and (`Gr Liv Area` > 2463.0999999999995) and (`Year Built` >= 1993.0),
 (`Overall Qual` <= 8.0) and (`Gr Liv Area` <= 2463.0999999999995) and (`BsmtFin SF 1` >= 1274.0)]

In [190]:
prism.plot_tree(final_rules)

NameError: name 'final_rules' is not defined

In [ ]:
X.query("`Garage Type`.isna()")

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
27,28,527425090,20,RL,70.0,10500,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,ConLI,Normal
119,120,534276360,20,RL,77.0,9320,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Shed,400,1,2010,WD,Normal
125,126,534427010,90,RL,98.0,13260,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,Oth,Abnorml
129,130,534450180,20,RL,50.0,7207,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
130,131,534451150,30,RL,55.0,5350,Pave,NaN,IR1,Lvl,...,0,0,NaN,GdWo,Shed,450,5,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,2914,923226180,180,RM,21.0,1470,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal
2916,2917,923228130,180,RM,21.0,1533,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,8,2006,WD,Abnorml
2918,2919,923228210,160,RM,21.0,1526,Pave,NaN,Reg,Lvl,...,0,0,NaN,GdPrv,NaN,0,6,2006,WD,Normal
2919,2920,923228260,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal


In [ ]:
X.query("`Garage Type`.notnull()")

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,2925,923251180,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,...,0,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,...,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal
2928,2929,924100070,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal
